In [1]:
from pydantic import BaseModel, ValidationError, field_validator, Field, ValidationInfo

In [2]:
class Model(BaseModel):
    field_1: int
    field_2: list[int]
    field_3: str
    field_4: list[str]

    @field_validator("field_3")
    @classmethod
    def validator(cls, value: str, validated_values: ValidationInfo):
        print(f"{value=}")
        print(f"{validated_values=}")
        return value

In [3]:
Model(field_1=100, field_2=[1, 2, 3], field_3="Python", field_4=list("pyThon"))

value='Python'
validated_values=ValidationInfo(config={'title': 'Model'}, context=None, data={'field_1': 100, 'field_2': [1, 2, 3]}, field_name='field_3')


Model(field_1=100, field_2=[1, 2, 3], field_3='Python', field_4=['p', 'y', 'T', 'h', 'o', 'n'])

In [4]:
try:
    Model(field_1=100, field_2=['a', 'v'], field_3="python", field_4=['a', ['b']])
except ValidationError as ex:
    print(ex)

value='python'
validated_values=ValidationInfo(config={'title': 'Model'}, context=None, data={'field_1': 100}, field_name='field_3')
3 validation errors for Model
field_2.0
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing
field_2.1
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='v', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing
field_4.1
  Input should be a valid string [type=string_type, input_value=['b'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [8]:
from pydantic import BaseModel, field_validator, ValidationError
from typing import Any, Annotated
import pytz
from dateutil.parser import parse
from datetime import datetime

In [9]:
from pydantic import BeforeValidator, AfterValidator

In [10]:

def parse_datetime(value: Any):
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValidationError(str(ex))
    return value


def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    
    return dt
        
DateTimeUTC = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(make_utc)]


In [14]:
class Model(BaseModel):
    start_dt: DateTimeUTC
    end_dt: DateTimeUTC

    @field_validator("end_dt")
    @classmethod
    def validate_end_date_after_start_date(cls, value: datetime, values: ValidationInfo):
        data = values.data # Dictionary
        if "start_dt" in data:
            if value <= data["start_dt"]:
                raise ValueError("end_dt must come after strt_dt")
        return value
 

In [15]:
Model(start_dt="2020/1/1", end_dt="2020/1/5")

Model(start_dt=datetime.datetime(2020, 1, 1, 0, 0, tzinfo=<UTC>), end_dt=datetime.datetime(2020, 1, 5, 0, 0, tzinfo=<UTC>))

In [16]:
try:
    Model(start_dt="2020/1/1", end_dt="2019/1/1")
except ValidationError as ex:
    print(ex)

1 validation error for Model
end_dt
  Value error, end_dt must come after strt_dt [type=value_error, input_value='2019/1/1', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
